In [313]:
import numpy as np
import pandas as pd
import seaborn as sns

In [314]:
train = pd.read_csv("../Datasets/Titanic/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [315]:
test = pd.read_csv("../Datasets/Titanic/test.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [316]:
train.shape
test.shape

(418, 11)

In [317]:
train = train.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])
test = test.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])

In [318]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non

In [319]:
train.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

What we've noticed here is that there are a bunch of null values for age so we need to find a way to deal with those. My plan is to simply remove the data that isn't valuable rather than trying to interpolate.

In [320]:
train["Age"] = train["Age"].replace(np.NaN, 25)
test["Age"] = test["Age"].replace(np.NaN, 25)
train.dropna()
test.dropna()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,25.0,0,0,8.0500,S
414,1,female,39.0,0,0,108.9000,C
415,3,male,38.5,0,0,7.2500,S
416,3,male,25.0,0,0,8.0500,S


In [321]:
train.isna().sum()
test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        1
Embarked    0
dtype: int64

In [329]:
train = train[train.Fare != np.NaN]
test = test[train.Fare != np.NaN]
train.head()
train.isna().sum()

C:\Users\kmccabewild\AppData\Local\Temp\ipykernel_10420\433587817.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = test[train.Fare != np.NaN]


Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

New issue is that we can't figure out what a 'female' is so we'll have to convert male and female to binary.
Male - 0     Female - 1

In [323]:
train["Sex"] = train["Sex"].replace('female', 1)
train["Sex"] = train["Sex"].replace('male', 0)

train["Embarked"] = train["Embarked"].replace('C', 0)
train["Embarked"] = train["Embarked"].replace('S', 1)
train["Embarked"] = train["Embarked"].replace('Q', 2)

In [324]:
from sklearn.model_selection import train_test_split

x_train = train.iloc[:, 1:]
y_train = train.iloc[:, :1]

In [325]:
from sklearn.ensemble import RandomForestRegressor
rgr = RandomForestRegressor()

In [326]:
rgr.fit(x_train, y_train)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

Now I'm going to fix all the test data :D

In [ ]:
test["Sex"] = test["Sex"].replace('female', 1)
test["Sex"] = test["Sex"].replace('male', 0)

test["Embarked"] = test["Embarked"].replace('C', 0)
test["Embarked"] = test["Embarked"].replace('S', 1)
test["Embarked"] = test["Embarked"].replace('Q', 2)

test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,1
2,2,0,62.0,0,0,9.6875,2
3,3,0,27.0,0,0,8.6625,1
4,3,1,22.0,1,1,12.2875,1


In [ ]:
x_test = test.iloc[:, :]
gs = pd.read_csv('../Datasets/Titanic/gender_submission.csv')
y_test = gs.iloc[:331, 1:]

In [ ]:
pred = rgr.predict(x_test)

In [ ]:
print(pred.shape)
print(y_test.shape)

(331,)
(331, 1)


In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(pred, y_test)

print(r2)

-1.5955411209180084
